<a href="https://colab.research.google.com/github/Hillascher5/nlp-tweets-sentiment-analysis/blob/main/Text_Classification_Tweets_Sentiment_no_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **NLP - Text Classification Project**
Group H - August 2025

Classification of tweets from Twitter that have been manually tagged for sentiment analysis.

In [71]:
# # Needed for Google Colab
# !pip install --quiet evaluate transformers optuna datasets nltk scikit-learn
# !pip install numpy==1.26.4

In [72]:
%env CUDA_LAUNCH_BLOCKING=1

from wordcloud import WordCloud, STOPWORDS
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from tqdm import tqdm
from collections import Counter
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer
from datasets import Dataset
from optuna.pruners import MedianPruner
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import string
import time
import glob
import nltk
import evaluate
import transformers
import optuna
import wandb
wandb.login()

os.environ["WANDB_PROJECT"] = "tweet-sentiment-classification"
os.environ["WANDB_INIT_TIMEOUT"] = "180"

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

env: CUDA_LAUNCH_BLOCKING=1


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [73]:
# Load data
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nlp_project/Data/Corona_NLP_train.csv', encoding='latin1')
df_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nlp_project/Data/Corona_NLP_test.csv', encoding='latin1')

### Pre-processing the Data

The tweets were cleaned by lowercasing (to reduce redundancy), removing stopwords, punctuation, numbers, short words, and applying lemmatization to reduce words to their base form (e.g. running → run). <br>This helps reduce noise and improve model performance.


In [74]:
# # Try agressive pre-processing
# lemmatizer = WordNetLemmatizer()

# def preprocess_text(text):
#     text = str(text).lower()
#     text = re.sub(r"http\S+|www.\S+", "", text)              # Remove URLs
#     text = re.sub(r"[^a-z\s]", "", text)                     # Remove punctuation & numbers
#     tokens = text.split()
#     tokens = [
#         lemmatizer.lemmatize(word)
#         for word in tokens
#         if word not in stop_words
#         and word not in domain_stopwords
#         and len(word) > 2
#     ]
#     return " ".join(tokens)

# df_train["clean_text"] = df_train["OriginalTweet"].apply(preprocess_text)
# df_val["clean_text"] = df_val["OriginalTweet"].apply(preprocess_text)

In [75]:
# # Try lighter preprocessing
# def light_preprocess(text):
#     text = re.sub(r"http\S+|www.\S+", "", text)   # Remove URLs
#     text = re.sub(r"@\w+", "", text)              # Remove @mentions
#     text = re.sub(r"#(\w+)", r"\1", text)         # Unwrap hashtags
#     return text.strip()
#
# df_train["clean_text"] = df_train["OriginalTweet"].apply(light_preprocess)
# df_val["clean_text"] = df_val["OriginalTweet"].apply(light_preprocess)

In [76]:
# Try without pre-processing
df_train["clean_text"] = df_train["OriginalTweet"]
df_val["clean_text"] = df_val["OriginalTweet"]

## Fine-Tuning Pretrained Language Models

Apply NLP techniques using transfer learning on our tweets dataset. Specifically, fine-tuning two pretrained transformer-based models from the Hugging Face library — BERT and RoBERTa — on our sentiment classification task. These models will be trained using both standard PyTorch and the Hugging Face API. Model performance will be monitored and tuned using hyperparameter optimization (Optuna) and experiment tracking (Weights & Biases).

**Load Pretrained Models**

Initialize tokenizers and models for BERT and RoBERTa, both widely used transformer architectures for text classification. The classification head is configured based on the number of sentiment labels.

In [77]:
# Choose pretrained models
bert_model_name = "bert-base-uncased"
roberta_model_name = "roberta-base"

sentiment_labels = df_train['Sentiment'].unique()
n_labels = len(sentiment_labels)

# Load BERT tokenizer and model
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
bert_model = AutoModelForSequenceClassification.from_pretrained(bert_model_name, num_labels=n_labels)

# Load RoBERTa tokenizer and model
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_model_name)
roberta_model = AutoModelForSequenceClassification.from_pretrained(roberta_model_name, num_labels=n_labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Encode Sentiment Labels**

Map each unique sentiment label to a numeric ID for model compatibility, and apply this mapping to both training and validation datasets.

In [78]:
# Mapping sentiments to unique numeric IDs
unique_labels = sorted(df_train["Sentiment"].unique())

label2id = {label: idx for idx, label in enumerate(unique_labels)}
id2label = {idx: label for label, idx in label2id.items()}

df_train["label"] = df_train["Sentiment"].map(label2id)
df_val["label"] = df_val["Sentiment"].map(label2id)

In [79]:
# Create stratified subset for Optuna
train_subset_df, _ = train_test_split(
    df_train[["clean_text", "label"]],
    train_size=2000,
    stratify=df_train["label"],
    random_state=42
    )
val_subset_df, _ = train_test_split(
    df_val[["clean_text", "label"]],
    train_size=500,
    stratify=df_val["label"],
    random_state=42
    )

**Tokenization**

Define tokenization functions for BERT and RoBERTa to preprocess text with truncation and fixed padding.

Transform training and validation DataFrames into Dataset objects compatible with Hugging Face workflows.

Apply tokenization to training and validation datasets using each model's tokenizer.

In [80]:
# Tokenize function
def tokenize_function_bert(examples):
    return bert_tokenizer(examples["clean_text"], truncation=True, padding='max_length', max_length=64)

def tokenize_function_roberta(examples):
    return roberta_tokenizer(examples["clean_text"], truncation=True, padding='max_length', max_length=64)

In [81]:
# Convert DataFrame to Hugging Face Dataset
hf_dataset_train = Dataset.from_pandas(df_train[["clean_text", "label"]])
hf_dataset_val = Dataset.from_pandas(df_val[["clean_text", "label"]])

hf_subset_train = Dataset.from_pandas(train_subset_df)
hf_subset_val = Dataset.from_pandas(val_subset_df)

In [82]:
# Tokenize subsets
# Tokenize for BERT
tokenized_bert_train_sub = hf_subset_train.map(tokenize_function_bert, batched=True)
tokenized_bert_train_sub.set_format("torch", columns=["input_ids", "attention_mask", "label"])

tokenized_bert_val_sub = hf_subset_val.map(tokenize_function_bert, batched=True)
tokenized_bert_val_sub.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# Tokenize for RoBERTa
tokenized_roberta_train_sub = hf_subset_train.map(tokenize_function_roberta, batched=True)
tokenized_roberta_train_sub.set_format("torch", columns=["input_ids", "attention_mask", "label"])

tokenized_roberta_val_sub = hf_subset_val.map(tokenize_function_roberta, batched=True)
tokenized_roberta_val_sub.set_format("torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [83]:
# Tokenize full dataset
# Tokenize for BERT
tokenized_bert_train = hf_dataset_train.map(tokenize_function_bert, batched=True)
tokenized_bert_train.set_format("torch", columns=["input_ids", "attention_mask", "label"])

tokenized_bert_val = hf_dataset_val.map(tokenize_function_bert, batched=True)
tokenized_bert_val.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# Tokenize for RoBERTa
tokenized_roberta_train = hf_dataset_train.map(tokenize_function_roberta, batched=True)
tokenized_roberta_train.set_format("torch", columns=["input_ids", "attention_mask", "label"])

tokenized_roberta_val = hf_dataset_val.map(tokenize_function_roberta, batched=True)
tokenized_roberta_val.set_format("torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/41157 [00:00<?, ? examples/s]

Map:   0%|          | 0/3798 [00:00<?, ? examples/s]

Map:   0%|          | 0/41157 [00:00<?, ? examples/s]

Map:   0%|          | 0/3798 [00:00<?, ? examples/s]

In [84]:
# # Define evaluation metric, Set up an evaluation function using accuracy as the metric to assess model performance during training.

# accuracy_metric = evaluate.load("accuracy")

# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=1)
#     return accuracy_metric.compute(predictions=predictions, references=labels)

In [85]:
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"],
        "f1_macro": f1_metric.compute(predictions=predictions, references=labels, average="macro")["f1"]
    }

**Use Small Subsets for Quick Evaluation**

Select shuffled samples from each training and validation dataset for both BERT and RoBERTa. This allows faster experimentation during model development.

In [86]:
USE_SMALL_DATASET = True
# if USE_SMALL_DATASET:
#   n_samples_train = 2000
#   n_samples_val = 500

#   train_dataset_bert = tokenized_bert_train.shuffle(seed=42).select(range(n_samples_train))
#   val_dataset_bert = tokenized_bert_val.shuffle(seed=42).select(range(n_samples_val))

#   train_dataset_roberta = tokenized_roberta_train.shuffle(seed=42).select(range(n_samples_train))
#   val_dataset_roberta = tokenized_roberta_val.shuffle(seed=42).select(range(n_samples_val))

# else:
#   n_samples_train = len(tokenized_bert_train)
#   n_samples_val = len(tokenized_bert_val)

#   train_dataset_bert = tokenized_bert_train
#   val_dataset_bert = tokenized_bert_val

#   train_dataset_roberta = tokenized_roberta_train
#   val_dataset_roberta = tokenized_roberta_val

**Define Training Arguments**

Configure hyperparameters and training settings for both BERT and RoBERTa models, including batch size, number of epochs, learning rate, and evaluation strategy.

In [87]:
# # Define hyperparameters
# run_time = f"run-{int(time.time())}"

# num_epochs_bert = 5
# learning_rate_bert = 4.037872385196561e-05
# batch_size_bert = 16

# bert_training_args = TrainingArguments(
#     output_dir="./results/bert/{run_time}",
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     per_device_train_batch_size=batch_size_bert,
#     per_device_eval_batch_size=batch_size_bert,
#     num_train_epochs=num_epochs_bert,
#     learning_rate=learning_rate_bert,
#     weight_decay=0.01,
#     load_best_model_at_end=True,
#     logging_dir="./logs/bert/{run_time}",
#     run_name = f"bert-ep{num_epochs_bert}-lr{learning_rate_bert}-bs{batch_size_bert}-samples{n_samples_train}-run{int(time.time())}-no_preprocess",
#     report_to="wandb"
# )

In [88]:
# # Define hyperparameters
# num_epochs_roberta = 4
# learning_rate_roberta = 4e-5
# batch_size_roberta = 16

# roberta_training_args = TrainingArguments(
#     output_dir="./results/roberta/{run_time}",
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     per_device_train_batch_size=batch_size_roberta,
#     per_device_eval_batch_size=batch_size_roberta,
#     num_train_epochs=num_epochs_roberta,
#     learning_rate=learning_rate_roberta,
#     weight_decay=0.01,
#     load_best_model_at_end=True,
#     logging_dir="./logs/roberta/{run_time}",
#     run_name=f"roberta-ep{num_epochs_roberta}-lr{learning_rate_roberta}-bs{batch_size_roberta}-samples{n_samples_train}-run{int(time.time())}-no_preprocess",
#     report_to="wandb"
# )

**Initialize Trainers**

Configure Trainer objects for both BERT and RoBERTa using the small training and validation sets, tokenizers, training arguments, and evaluation metric.

In [89]:
# bert_trainer = Trainer(
#     model=bert_model,
#     args=bert_training_args,
#     train_dataset=train_dataset_bert,
#     eval_dataset=val_dataset_bert,
#     tokenizer=bert_tokenizer,
#     compute_metrics=compute_metrics
# )

In [90]:
# roberta_trainer = Trainer(
#     model=roberta_model,
#     args=roberta_training_args,
#     train_dataset=train_dataset_roberta,
#     eval_dataset=val_dataset_roberta,
#     tokenizer=roberta_tokenizer,
#     compute_metrics=compute_metrics
# )

**Model Training**

Train both BERT and RoBERTa models on the small datasets using the Trainer interface.

## Manual Trials

Manually trained BERT and RoBERTa on various sample sizes to understand how accuracy scales:
- 1000, 5000, 10000 train samples for 5 epochs

This section is useful for baseline comparison before running Optuna for automated tuning.


In [91]:
# bert_trainer.train()

In [92]:
# roberta_trainer.train()

# # Safely finish run with specific parameters
# wandb.finish()

**Hyperparameters Tuning with Optuna**

In [93]:
def build_trainer(model_checkpoint, trial, run_prefix, train_dataset, val_dataset):
    # Sample hyperparameters
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    num_epochs = trial.suggest_int("num_train_epochs", 2, 5)
    n_samples = len(train_dataset)

    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=5)
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

    run_name = f"{run_prefix}-ep{num_epochs}-lr{learning_rate}-bs{batch_size}-samples{n_samples}-run{int(time.time())}-no_preprocess"

    args = TrainingArguments(
        output_dir=f"./results/{run_prefix}/{run_name}",
        disable_tqdm=True,
        fp16=True,
        eval_strategy="epoch",
        save_strategy="epoch",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        learning_rate=learning_rate,
        weight_decay=0.01,
        load_best_model_at_end=True,
        save_total_limit=1,
        logging_strategy="epoch",
        logging_dir=f"./logs/{run_prefix}/{run_name}",
        run_name=run_name,
        report_to="wandb",
        metric_for_best_model="f1_macro",
        greater_is_better=True
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    return trainer

In [94]:
def objective_bert(trial):
    trainer = build_trainer(
        model_checkpoint="bert-base-uncased",
        trial=trial,
        run_prefix="bert",
        train_dataset=tokenized_bert_train_sub,
        val_dataset=tokenized_bert_val_sub
    )
    trainer.train()
    eval_result = trainer.evaluate()
    return eval_result["eval_f1_macro"]

def objective_roberta(trial):
    trainer = build_trainer(
        model_checkpoint="roberta-base",
        trial=trial,
        run_prefix="roberta",
        train_dataset=tokenized_roberta_train_sub,
        val_dataset=tokenized_roberta_val_sub
    )
    trainer.train()
    eval_result = trainer.evaluate()
    return eval_result["eval_f1_macro"]

In [95]:
study_bert = optuna.create_study(direction="maximize",
                                 pruner=MedianPruner(n_startup_trials=2, n_warmup_steps=1),
                                 study_name="bert_study_stratify_no_preprocess",
                                 storage="sqlite:////content/drive/MyDrive/Colab Notebooks/nlp_project/optuna/bert_study_stratify_no_preprocess.db",
                                 load_if_exists=True)
study_bert.optimize(objective_bert, n_trials=5)

[I 2025-07-27 20:49:10,660] Using an existing study with name 'bert_study_stratify_no_preprocess' instead of creating a new one.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-93-2841737310.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


{'loss': 1.473, 'grad_norm': 11.279875755310059, 'learning_rate': 2.06462268431497e-05, 'epoch': 1.0}
{'eval_loss': 1.2267123460769653, 'eval_accuracy': 0.462, 'eval_f1_macro': 0.4735340832257668, 'eval_runtime': 1.3649, 'eval_samples_per_second': 366.332, 'eval_steps_per_second': 46.158, 'epoch': 1.0}
{'loss': 0.9946, 'grad_norm': 16.656282424926758, 'learning_rate': 3.2642255878497555e-07, 'epoch': 2.0}
{'eval_loss': 1.0689821243286133, 'eval_accuracy': 0.57, 'eval_f1_macro': 0.5811683970332522, 'eval_runtime': 1.3784, 'eval_samples_per_second': 362.745, 'eval_steps_per_second': 45.706, 'epoch': 2.0}
{'train_runtime': 45.4665, 'train_samples_per_second': 87.977, 'train_steps_per_second': 10.997, 'train_loss': 1.2337723388671875, 'epoch': 2.0}
{'eval_loss': 1.0689821243286133, 'eval_accuracy': 0.57, 'eval_f1_macro': 0.5811683970332522, 'eval_runtime': 1.4236, 'eval_samples_per_second': 351.213, 'eval_steps_per_second': 44.253, 'epoch': 2.0}


[I 2025-07-27 20:50:00,286] Trial 5 finished with value: 0.5811683970332522 and parameters: {'learning_rate': 4.080281984812194e-05, 'batch_size': 8, 'num_train_epochs': 2}. Best is trial 2 with value: 0.6268387639451145.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-93-2841737310.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


{'loss': 1.5671, 'grad_norm': 6.147643089294434, 'learning_rate': 1.749415907768789e-05, 'epoch': 1.0}
{'eval_loss': 1.5139579772949219, 'eval_accuracy': 0.302, 'eval_f1_macro': 0.18981580214158172, 'eval_runtime': 0.4438, 'eval_samples_per_second': 1126.683, 'eval_steps_per_second': 36.054, 'epoch': 1.0}
{'loss': 1.4454, 'grad_norm': 8.56684398651123, 'learning_rate': 1.3207052900546984e-05, 'epoch': 2.0}
{'eval_loss': 1.4393876791000366, 'eval_accuracy': 0.368, 'eval_f1_macro': 0.29760501599783185, 'eval_runtime': 0.4183, 'eval_samples_per_second': 1195.443, 'eval_steps_per_second': 38.254, 'epoch': 2.0}
{'loss': 1.2512, 'grad_norm': 10.294244766235352, 'learning_rate': 8.850799849581226e-06, 'epoch': 3.0}
{'eval_loss': 1.3034530878067017, 'eval_accuracy': 0.416, 'eval_f1_macro': 0.42343033545198805, 'eval_runtime': 0.4252, 'eval_samples_per_second': 1176.024, 'eval_steps_per_second': 37.633, 'epoch': 3.0}
{'loss': 1.0714, 'grad_norm': 16.660293579101562, 'learning_rate': 4.563693672

[I 2025-07-27 20:50:43,636] Trial 6 finished with value: 0.4672332860944926 and parameters: {'learning_rate': 2.1781265254828797e-05, 'batch_size': 32, 'num_train_epochs': 5}. Best is trial 2 with value: 0.6268387639451145.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-93-2841737310.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


...

{'loss': 1.5648, 'grad_norm': 30.777372360229492, 'learning_rate': 1.0671293901080498e-05, 'epoch': 1.0}
{'eval_loss': 1.4943945407867432, 'eval_accuracy': 0.362, 'eval_f1_macro': 0.3574108026484007, 'eval_runtime': 1.3813, 'eval_samples_per_second': 361.972, 'eval_steps_per_second': 45.608, 'epoch': 1.0}
{'loss': 1.2817, 'grad_norm': 8.48298168182373, 'learning_rate': 2.0924105688393135e-07, 'epoch': 2.0}
{'eval_loss': 1.2750639915466309, 'eval_accuracy': 0.456, 'eval_f1_macro': 0.4706886475858491, 'eval_runtime': 1.3667, 'eval_samples_per_second': 365.846, 'eval_steps_per_second': 46.097, 'epoch': 2.0}
{'train_runtime': 45.3483, 'train_samples_per_second': 88.206, 'train_steps_per_second': 11.026, 'train_loss': 1.4232604370117188, 'epoch': 2.0}
{'eval_loss': 1.2750639915466309, 'eval_accuracy': 0.456, 'eval_f1_macro': 0.4706886475858491, 'eval_runtime': 1.3888, 'eval_samples_per_second': 360.026, 'eval_steps_per_second': 45.363, 'epoch': 2.0}


[I 2025-07-27 20:52:44,180] Trial 8 finished with value: 0.4706886475858491 and parameters: {'learning_rate': 2.0924105688393135e-05, 'batch_size': 8, 'num_train_epochs': 2}. Best is trial 2 with value: 0.6268387639451145.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-93-2841737310.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


{'loss': 1.4887, 'grad_norm': 7.255120277404785, 'learning_rate': 1.4302892789266096e-05, 'epoch': 1.0}
{'eval_loss': 1.333715796470642, 'eval_accuracy': 0.434, 'eval_f1_macro': 0.44481439325718847, 'eval_runtime': 0.7338, 'eval_samples_per_second': 681.373, 'eval_steps_per_second': 43.608, 'epoch': 1.0}
{'loss': 1.128, 'grad_norm': 8.103170394897461, 'learning_rate': 3.3522404974842415e-07, 'epoch': 2.0}
{'eval_loss': 1.242388129234314, 'eval_accuracy': 0.468, 'eval_f1_macro': 0.4787755848748746, 'eval_runtime': 0.7591, 'eval_samples_per_second': 658.665, 'eval_steps_per_second': 42.155, 'epoch': 2.0}
{'train_runtime': 25.5296, 'train_samples_per_second': 156.681, 'train_steps_per_second': 9.793, 'train_loss': 1.3083780517578125, 'epoch': 2.0}


[I 2025-07-27 20:53:12,027] Trial 9 finished with value: 0.4787755848748746 and parameters: {'learning_rate': 2.7935337479035344e-05, 'batch_size': 16, 'num_train_epochs': 2}. Best is trial 2 with value: 0.6268387639451145.


{'eval_loss': 1.242388129234314, 'eval_accuracy': 0.468, 'eval_f1_macro': 0.4787755848748746, 'eval_runtime': 0.7718, 'eval_samples_per_second': 647.851, 'eval_steps_per_second': 41.462, 'epoch': 2.0}


In [96]:
study_roberta = optuna.create_study(direction="maximize",
                                    pruner=MedianPruner(n_startup_trials=2, n_warmup_steps=1),
                                    study_name="roberta_study_stratify_no_preprocess",
                                    storage="sqlite:////content/drive/MyDrive/Colab Notebooks/nlp_project/optuna/roberta_study_stratify_no_preprocess.db",
                                    load_if_exists=True)
study_roberta.optimize(objective_roberta, n_trials=5)

[I 2025-07-27 20:53:12,074] Using an existing study with name 'roberta_study_stratify_no_preprocess' instead of creating a new one.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-93-2841737310.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


{'loss': 1.5593, 'grad_norm': 7.488118648529053, 'learning_rate': 1.9870364087802768e-05, 'epoch': 1.0}
{'eval_loss': 1.4581249952316284, 'eval_accuracy': 0.376, 'eval_f1_macro': 0.31566675152522583, 'eval_runtime': 0.4292, 'eval_samples_per_second': 1164.982, 'eval_steps_per_second': 37.279, 'epoch': 1.0}
{'loss': 1.3022, 'grad_norm': 13.697330474853516, 'learning_rate': 1.3316264938422798e-05, 'epoch': 2.0}
{'eval_loss': 1.2754279375076294, 'eval_accuracy': 0.44, 'eval_f1_macro': 0.45286557390714843, 'eval_runtime': 0.4195, 'eval_samples_per_second': 1191.884, 'eval_steps_per_second': 38.14, 'epoch': 2.0}
{'loss': 1.0645, 'grad_norm': 22.469097137451172, 'learning_rate': 6.866199108874256e-06, 'epoch': 3.0}
{'eval_loss': 1.261343240737915, 'eval_accuracy': 0.446, 'eval_f1_macro': 0.4570951305191041, 'eval_runtime': 0.4146, 'eval_samples_per_second': 1206.085, 'eval_steps_per_second': 38.595, 'epoch': 3.0}
{'loss': 0.9009, 'grad_norm': 41.303890228271484, 'learning_rate': 3.1209995949

[I 2025-07-27 20:53:50,535] Trial 5 finished with value: 0.4979689975080307 and parameters: {'learning_rate': 2.6216396597519885e-05, 'batch_size': 32, 'num_train_epochs': 4}. Best is trial 3 with value: 0.5954185429373753.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-93-2841737310.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


{'loss': 1.5876, 'grad_norm': 2.97064208984375, 'learning_rate': 3.287939618494191e-05, 'epoch': 1.0}
{'eval_loss': 1.5830800533294678, 'eval_accuracy': 0.274, 'eval_f1_macro': 0.08602825745682888, 'eval_runtime': 0.7366, 'eval_samples_per_second': 678.833, 'eval_steps_per_second': 43.445, 'epoch': 1.0}
{'loss': 1.5702, 'grad_norm': 2.9269001483917236, 'learning_rate': 2.474158156032555e-05, 'epoch': 2.0}
{'eval_loss': 1.5577460527420044, 'eval_accuracy': 0.278, 'eval_f1_macro': 0.1574894765055847, 'eval_runtime': 0.7203, 'eval_samples_per_second': 694.14, 'eval_steps_per_second': 44.425, 'epoch': 2.0}
{'loss': 1.5483, 'grad_norm': 2.9746205806732178, 'learning_rate': 1.6669394473004482e-05, 'epoch': 3.0}
{'eval_loss': 1.5391367673873901, 'eval_accuracy': 0.306, 'eval_f1_macro': 0.19329253639589794, 'eval_runtime': 0.7312, 'eval_samples_per_second': 683.807, 'eval_steps_per_second': 43.764, 'epoch': 3.0}
{'loss': 1.5385, 'grad_norm': 5.046313285827637, 'learning_rate': 8.46595231109282

[I 2025-07-27 20:54:59,413] Trial 6 finished with value: 0.19329253639589794 and parameters: {'learning_rate': 4.101721080955828e-05, 'batch_size': 16, 'num_train_epochs': 5}. Best is trial 3 with value: 0.5954185429373753.


...

{'eval_loss': 1.3055121898651123, 'eval_accuracy': 0.41, 'eval_f1_macro': 0.42351176134457863, 'eval_runtime': 0.7715, 'eval_samples_per_second': 648.054, 'eval_steps_per_second': 41.475, 'epoch': 3.0}


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-93-2841737310.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


{'loss': 1.5812, 'grad_norm': 4.143873691558838, 'learning_rate': 5.964058131372879e-06, 'epoch': 1.0}
{'eval_loss': 1.5650722980499268, 'eval_accuracy': 0.256, 'eval_f1_macro': 0.09381363244176014, 'eval_runtime': 0.4207, 'eval_samples_per_second': 1188.385, 'eval_steps_per_second': 38.028, 'epoch': 1.0}
{'loss': 1.5305, 'grad_norm': 5.970037460327148, 'learning_rate': 2.75264221447979e-07, 'epoch': 2.0}
{'eval_loss': 1.534778356552124, 'eval_accuracy': 0.286, 'eval_f1_macro': 0.17301249740797733, 'eval_runtime': 0.4235, 'eval_samples_per_second': 1180.75, 'eval_steps_per_second': 37.784, 'epoch': 2.0}
{'train_runtime': 19.7568, 'train_samples_per_second': 202.462, 'train_steps_per_second': 6.378, 'train_loss': 1.555830334860181, 'epoch': 2.0}


[I 2025-07-27 20:57:02,829] Trial 9 finished with value: 0.17301249740797733 and parameters: {'learning_rate': 1.1561097300815119e-05, 'batch_size': 32, 'num_train_epochs': 2}. Best is trial 3 with value: 0.5954185429373753.


{'eval_loss': 1.534778356552124, 'eval_accuracy': 0.286, 'eval_f1_macro': 0.17301249740797733, 'eval_runtime': 0.4346, 'eval_samples_per_second': 1150.495, 'eval_steps_per_second': 36.816, 'epoch': 2.0}


In [97]:
best_trial_bert = study_bert.best_trial
print('Bert best trial on subset:')
print(best_trial_bert.params)
best_trial_roberta = study_roberta.best_trial
print('RoBerta best trial on subset:')
print(best_trial_roberta.params)

Bert best trial on subset:
{'learning_rate': 3.793274955606689e-05, 'batch_size': 32, 'num_train_epochs': 5}
RoBerta best trial on subset:
{'learning_rate': 2.2755931315738675e-05, 'batch_size': 8, 'num_train_epochs': 5}


In [98]:
final_trainer_bert = build_trainer(
    model_checkpoint="bert-base-uncased",
    trial=best_trial_bert,
    run_prefix="bert_final_stratify_no_preprocess",
    train_dataset=tokenized_bert_train,
    val_dataset=tokenized_bert_val
)
final_trainer_bert.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-93-2841737310.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


{'loss': 0.7267, 'grad_norm': 31.37412452697754, 'learning_rate': 3.036977866555658e-05, 'epoch': 1.0}
{'eval_loss': 0.5666289329528809, 'eval_accuracy': 0.8059505002632965, 'eval_f1_macro': 0.8055703175499342, 'eval_runtime': 3.0648, 'eval_samples_per_second': 1239.217, 'eval_steps_per_second': 38.827, 'epoch': 1.0}
{'loss': 0.3673, 'grad_norm': 6.351016521453857, 'learning_rate': 2.2783228754343205e-05, 'epoch': 2.0}
{'eval_loss': 0.5646061301231384, 'eval_accuracy': 0.8085834649815693, 'eval_f1_macro': 0.8151192242858502, 'eval_runtime': 3.0615, 'eval_samples_per_second': 1240.583, 'eval_steps_per_second': 38.87, 'epoch': 2.0}
{'loss': 0.2431, 'grad_norm': 20.84602928161621, 'learning_rate': 1.5202573598305595e-05, 'epoch': 3.0}
{'eval_loss': 0.4836413860321045, 'eval_accuracy': 0.8504476040021064, 'eval_f1_macro': 0.855097155032054, 'eval_runtime': 3.02, 'eval_samples_per_second': 1257.605, 'eval_steps_per_second': 39.404, 'epoch': 3.0}
{'loss': 0.165, 'grad_norm': 16.0933399200439

TrainOutput(global_step=6435, training_loss=0.3215532138353183, metrics={'train_runtime': 598.5939, 'train_samples_per_second': 343.781, 'train_steps_per_second': 10.75, 'train_loss': 0.3215532138353183, 'epoch': 5.0})

In [99]:
final_trainer_roberta = build_trainer(
    model_checkpoint="roberta-base",
    trial=best_trial_roberta,
    run_prefix="roberta_final_stratify_no_preprocess",
    train_dataset=tokenized_roberta_train,
    val_dataset=tokenized_roberta_val
)
final_trainer_roberta.train()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-93-2841737310.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


{'loss': 0.8834, 'grad_norm': 20.14725685119629, 'learning_rate': 1.8210937142744882e-05, 'epoch': 1.0}
{'eval_loss': 0.9594243168830872, 'eval_accuracy': 0.667983149025803, 'eval_f1_macro': 0.6772403663898248, 'eval_runtime': 10.2745, 'eval_samples_per_second': 369.652, 'eval_steps_per_second': 46.231, 'epoch': 1.0}
{'loss': 0.5519, 'grad_norm': 15.50577449798584, 'learning_rate': 1.3662404632520267e-05, 'epoch': 2.0}
{'eval_loss': 0.6814717054367065, 'eval_accuracy': 0.7785676671932596, 'eval_f1_macro': 0.7869201780821529, 'eval_runtime': 10.5122, 'eval_samples_per_second': 361.294, 'eval_steps_per_second': 45.185, 'epoch': 2.0}
{'loss': 0.4362, 'grad_norm': 3.4310147762298584, 'learning_rate': 9.112987537987942e-06, 'epoch': 3.0}
{'eval_loss': 0.6444135904312134, 'eval_accuracy': 0.8048973143759873, 'eval_f1_macro': 0.8127646870659374, 'eval_runtime': 10.3861, 'eval_samples_per_second': 365.68, 'eval_steps_per_second': 45.734, 'epoch': 3.0}
{'loss': 0.3615, 'grad_norm': 125.06330871

TrainOutput(global_step=25725, training_loss=0.5041161350142735, metrics={'train_runtime': 2078.1942, 'train_samples_per_second': 99.021, 'train_steps_per_second': 12.379, 'train_loss': 0.5041161350142735, 'epoch': 5.0})

In [100]:
final_trainer_bert.save_model("models/bert_final_stratify_no_preprocess")
bert_tokenizer.save_pretrained("models/bert_final_stratify_no_preprocess")
!cp -r models/bert_final_stratify_no_preprocess "/content/drive/MyDrive/Colab Notebooks/nlp_project/models/bert_best_model_stratify_no_preprocess"

final_trainer_roberta.save_model("models/roberta_final_stratify_no_preprocess")
roberta_tokenizer.save_pretrained("models/roberta_final_stratify_no_preprocess")
!cp -r models/roberta_final_stratify_no_preprocess "/content/drive/MyDrive/Colab Notebooks/nlp_project/models/roberta_best_model_stratify_no_preprocess"